# Extract features

Here, we want to extract features from the seismic signals

In [ ]:
from tqdm.notebook import tqdm

import numpy as np
from obspy import ObsPyException, UTCDateTime

import analysis
import catalog
import energy

tqdm.pandas()
catalog.display_parameters()

## Load dataframe

In [ ]:
dataframe = catalog.load("data/new_dataframe_with_inventory_cleaned.pickle")

catalog.open_plot(dataframe, pos_number=0.04, xlim=1600)

## Extract features

After detecting the seismic signal with the detection method, features will be extracted from the spectrogram and the energy envelope of the signals.

In [ ]:
## Loop over all events
for event_index in tqdm(dataframe["numero"], total=len(dataframe)):
        
    #if event_index == 0: ## Uncomment this line if you want to see a specific event

        print("-------------------------")
        print("Event number", event_index)
        print("-------------------------")
        
        try:
            ## Filter the stream
            _, trace = analysis.filter_stream(dataframe, event_index, 0, freq_HP=9, freq_LP=0.5, max_percentage=0.3)
        
            # The detection method
            time_start_detection, data_start_detection, trimmed_time, trimmed_data, time_raw, data_raw, upper_threshold, lower_threshold = analysis.detection_on_one_trace(trace, dataframe, event_index)

            ## Plot the spectrum with the models and extract features in a dataframe
            analysis.plot_spectre(trace, dataframe, trimmed_data, 0, event_index, conserv_result=True)

            ## Compute the energy and extract features in a dataframe
            energy.compute(dataframe, trace, event_index)

        except (IndexError, ObsPyException, ValueError):
            print("Une erreur est survenue.")